## Import Packages and Set Starting Parameters

In [ ]:
import os, pandas as pd, slr_pkg.clean_load_data as cld, slr_pkg.para as para
from slr_pkg.clean_load_data import open_table
from pathlib import Path


# Get current working directory
bp = Path(os.getcwd())

dp = bp / 'data'

# Set sample data directory.
edf_path = dp / 'geotracker_edf_results'
gama_path = dp / 'gama_results'

# Set location data directory.
geo_xy_path = dp / 'geotracker_xy'
gama_xy_path = dp / "gama_xy"

# Set results directory
results_path = dp / "results"

# Create path to mcl table.
mcl_path = dp / 'MCLs.xlsx'

# Ask for county to gather data for.
# area = input('Enter county: ')
area = 'Kern'

# List of contaminants.
chems = para.conts11

## Open and Concat Data

### Load Sample Data

In [ ]:
edf_files = edf_path.glob('**/*{}*.zip'.format(area))
gama_files = gama_path.glob('**/*{}*.zip'.format(area.lower()))

samples = cld.Sample_Data.full_dataset(edf_files, gama_files)

print(samples)
print("Unique Sample IDs (SIDs):" ,len(samples['SID'].unique()))

### Load Location Data

In [ ]:
geo_xy_files = geo_xy_path.glob('**/*{}*.zip'.format(area))
gama_xy_files = gama_xy_path.glob('**/*.zip')

locations = cld.Location_Data.full_dataset(geo_xy_files, gama_xy_files)

print(locations)
print(len(locations['WID'].unique()))

### Join Sample and Location Data

In [ ]:
# Join well location data to sample results.
samples = samples.merge(locations, left_on='WID', right_on='WID', how='inner')
print(len(samples))
print(len(samples['SID'].unique()))

### Join MCL Table to Samples

In [ ]:
print('Loading MCL table \n')

# Create path to mcl table.
mcl_path = bp / 'MCLs.xlsx'

# Open mcl table.
mcl = pd.read_excel(mcl_path,sheet_name='MCL', engine='openpyxl')

# join MCL values to sample results
print('Joining MCL values to samples \n')
samples = samples.merge(mcl, left_on='PARLABEL', right_on='chem_abrv', how='inner')
print(len(samples))
print(len(samples['SID'].unique()))

### Join Unit Conversion Data to Samples

In [ ]:
# Load conversion tables.
metric_conversion = pd.read_excel(bp / 'unit_conversion.xlsx', sheet_name='metric')

# join coversion factors to samples based on sample unit.
samples = samples.merge(metric_conversion, how='inner', left_on='UNITS', right_on='start_unit')
print(len(samples))
print(len(samples['SID'].unique()))

### Convert Units

In [ ]:
# Create mask for samples with MCL units in UG/L and converts sample result units to UG/L.
mask = samples['UNITS'] != samples['units']

# Multiply sample results by conversion factor.
samples.loc[mask, 'PARVAL'] = samples['PARVAL'] * samples['coef']
samples['UNITS'] = 'UG/L'

# Drop columns that are not needed.
samples.drop(columns=['REPDL','chem_abrv', 'units','comp_conc_type','start_unit', 'coef'], inplace=True)
print(len(samples))
print(len(samples['SID'].unique()))

### Add Exceedence and Magnitude Attributes

In [ ]:
# Create exceedence attribute, true if sample result exceeds reporting limit.
samples['exceedence'] = samples['PARVAL'] > samples['comp_conc_val']

# Create magnitude attribute. Sample result value divided by the comparison concentration value (MCL or Action level) minus 1.
samples['magnitude'] = (samples['PARVAL'] / samples['comp_conc_val']) - 1

In [ ]:
print(len(samples))
print(len(samples['SID'].unique()))

### **All Samples**

In [ ]:
# samples.to_csv(results_path / '{}_all_sample_results.csv'.format(area.lower()))

### Select Specific Wells

In [ ]:
# subset of specific samples meeting parameters.
spec_samples = samples
print(len(spec_samples))
print(len(spec_samples['SID'].unique()))
# Select spec_samples taken since 2010.
spec_samples = spec_samples.loc[spec_samples['LOGDATE'] >= '2012-01-01']

# Select samples with contaminants of interest.
spec_samples = spec_samples.loc[spec_samples['PARLABEL'].isin(chems)]

# Create groups of spec_samples based on WID and PARLABEL(contaminant label).
sample_groups = spec_samples.groupby(['WID'])['PARLABEL'].apply(list).reset_index()

In [ ]:
print(len(spec_samples))
print(len(spec_samples['SID'].unique()))

In [ ]:
from collections import Counter


def select_wells(row):
    wid = row['WID']
    counter = Counter(row['PARLABEL'])
    if len(counter) == len(chems):
        print(counter.values())
        if all(i >= 4 for i in counter.values()):
            return  wid


# Create mask of sample groups meeting parameter requirements.
res = sample_groups.apply(select_wells, axis=1)

In [ ]:
# Use mask to select sample results from wells that meet parameter requirements.
spec_samples = spec_samples[spec_samples['WID'].isin(res)]

In [ ]:
print(len(samples))
print(len(samples['SID'].unique()))

In [ ]:
# number of unique wells
nwells = len(spec_samples['WID'].unique())
print('Number of wells: ' + str(nwells))

In [ ]:
# Save sample results to csv.
#spec_samples.to_csv(results_path / '{}_spec_sample_results_11.csv'.format(area.lower()))

### Join Groundwater Elevations to Sample Results

In [ ]:
### Depth to Water Data
# Load GAMA dtw data
# Create elev_path.
elev_path = bp / 'elevation'
print(elev_path, '\n')

# Dictionary of data types for gama_elev gama_elev for open_table().
gama_elev_dtypes = {
    'WELL NUMBER' : 'string',
    'DEPTH TO WATER' : 'float64',
    }

# Date column of gama_elev gama_elev for open_table().
gama_elev_date = ['MEASUREMENT DATE']

# Columns of gama_elev gama_elev for open_table().
gama_elev_cols = list(gama_elev_dtypes.keys()) + gama_elev_date


print('Loading GAMA groundwater elevations. \n')

# create list of files to open
gama_elev_files = elev_path.glob('**/*gama*.zip')
gama_elev_files = list(gama_elev_files)

# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.
gama_elev_list = [open_table(i,dtypes = gama_elev_dtypes,date_cols = gama_elev_date, cols =gama_elev_cols) for i in gama_elev_files]
print(gama_elev_list)

# Concatenate the list of dataframes into one dataframe if there are more than one.
if len(gama_elev_list) > 1:
    gama_elev = pd.concat(gama_elev_list)

else:
    gama_elev = gama_elev_list[0]

# Dict of attributes to rename.
gama_geo_dict = {
    'WELL NUMBER' : 'WID',
    'DEPTH TO WATER' : 'DTW',
    'MEASUREMENT DATE' : 'LOGDATE',
}
# Rename columns.
gama_elev = gama_elev.rename(columns=gama_geo_dict)

# Fix column formatting.
gama_elev['LOGDATE'] = gama_elev['LOGDATE'].astype(str)
gama_elev['WID'] = gama_elev['WID'].str.replace(' ', '')

# Create GID (group id) column. GID is the WID and LOGDATE concatenated.
gama_elev['GID'] = list(zip(gama_elev['WID'], gama_elev['LOGDATE']))
# Load Geotracker DTW data.
# Dictionary of data types for geo_elev geo_elev for open_table().
geo_elev_dtypes = {
    'GLOBAL_ID' : 'string',
    'FIELD_POINT_NAME' : 'string',
    'DTW' : 'float64',
    }

# Date column of geo_elev geo_elev for open_table().
geo_elev_date = ['GW_MEAS_DATE']

# Columns of geo_elev geo_elev for open_table().
geo_elev_cols = list(geo_elev_dtypes.keys()) + geo_elev_date

print('Loading Geotracker groundwater elevations. \n')

# create list of files to open
geo_elev_files = elev_path.glob('**/*Geo*.zip')
geo_elev_files = list(geo_elev_files)


# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.
geo_elev_list = [open_table(i,geo_elev_dtypes,date_cols= geo_elev_date,cols =geo_elev_cols) for i in geo_elev_files]

# Concatenate the list of dataframes into one dataframe if there are more than one.
if len(geo_elev_list) > 1:
    geo_elev = pd.concat(geo_elev_list)

else:
    geo_elev = geo_elev_list[0]

# Create WID column.
geo_elev['WID'] = geo_elev['GLOBAL_ID'] + '-' + geo_elev['FIELD_POINT_NAME']

# Drop unnecessary columns.
geo_elev = geo_elev.drop(columns=['GLOBAL_ID', 'FIELD_POINT_NAME'])

# fix column formatting.
geo_elev['WID'] = geo_elev['WID'].str.replace(' ', '')

# Rename columns.
geo_elev = geo_elev.rename(columns={'GW_MEAS_DATE' : 'LOGDATE'})

# Fix column formatting.
geo_elev['LOGDATE'] = geo_elev['LOGDATE'].astype(str)

# Create GID (group id) column. GID is the WID and LOGDATE concatenated.
geo_elev['GID'] = list(zip(geo_elev['WID'], geo_elev['LOGDATE']))

In [ ]:
# Concatenate gama_results and edf_results.
dtw = pd.concat([geo_elev, gama_elev])
dtw['dtw_units'] = 'ft'

# List of columns that require a value.
dtw_req_cols = ['WID','DTW','LOGDATE']

# Drops rows with missing values in required columns.
dtw = dtw.dropna(subset=dtw_req_cols)

# Drop duplicate GID rows.
dtw = dtw.drop_duplicates(subset=['GID'])
samples_dtw = spec_samples.merge(dtw, left_on=['GID'], right_on=['GID'], how='inner')

In [ ]:
spec_samples

In [ ]:
samples_dtw.columns

dtw_req_cols = ['LOGDATE_x', 'UNITS','PARLABEL', 'PARVAL', 'WID_x', 'DTW', 'LATITUDE', 'LONGITUDE', 'dtw_units']

for i in samples_dtw.columns:
    if i not in dtw_req_cols:
        samples_dtw = samples_dtw.drop(columns=i)

print(samples_dtw.columns)

In [ ]:
chem_num = str(len(chems))

a = (len(spec_samples))
b = (len(samples_dtw))
c =((len(samples_dtw) / len(spec_samples)*100))
c = "{:.2f}".format(c)

print(area, chem_num,': \n')
print('There are ' + str(b) + ' samples with depth to water values.')
print("Out of " + str(a) + " samples in the original dataframe.")
print(str(c) + "% of samples. \n")

a = (len(spec_samples['WID'].unique()))
b = (len(samples_dtw['WID_x'].unique()))
c = (b/a)*100
c = "{:.2f}".format(c)

print('There are ' + str(b) + ' wells with depth to water values.')
print("Out of " + str(a) + " wells in the original dataframe.")
print(str(c) + "% of  wells. \n")

In [ ]:
# Save sample results to csv.
samples_dtw.to_csv(results_path / '{}_dd_dtw_sample_results_{}.csv'.format(area.lower(), chem_num))

In [ ]:
# Run from here

### Pivot table for CCME Water Quality Index

In [ ]:
sample_results.rename(columns={'WID' : 'Station', 'LOGDATE' : 'Date'}, inplace=True)

sample_results['PARLABEL'] = sample_results['PARLABEL'] + '_' + sample_results['units']

pivot_table = pd.pivot_table(sample_results, index=['Station', 'Date'], columns=['PARLABEL'], values=['PARVAL'])
ccme_wqi_data = pivot_table.reset_index()

ccme_wqi_data.columns = ['Station', 'Date', 'AS_UG/L', 'BZME_UG/L', 'BZ_UG/L', 'CD_UG/L', 'DBCP_UG/L',
       'EBZ_UG/L', 'EDB_UG/L', 'MTBE_UG/L', 'NO3N_MG/L', 'PB_UG/L', 'PCE_UG/L',
       'TCE_UG/L', 'TCPR123_UG/L', 'THM_UG/L', 'XYLENES_UG/L']

ccme_wqi_data.dropna(inplace=True)

In [ ]:
ccme_wqi_data.to_csv(results_path / '{}_ccme_wqi_conc_samples.csv'.format(county.lower()))

### Normalize Sample Result Values at Wells

In [ ]:
# Calculates the mean of magnitudes for each WID in the exceedences dataframe.
print('Calculating magnitudes for each WID \n')
print(samples_mcl.head())

means = samples_mcl.groupby(['WID'])['magnitude'].mean()

In [ ]:
# Join mean magnitudes to well locations.
print('Merging geometric mean magnitudes to wells \n')
wells = wells.merge(means, how='inner', left_on='WID', right_index=True)
wells = wells.set_index('WID').sort_index()

# Save well mean magnitudes to csv.
wells.to_csv(bp / 'wells.csv')

In [ ]:
# Convert well mean magnitudes to shapefile
import geopandas as gpd

# Create geodataframe from well mean magnitudes, uses long and lat columns as xy coordinates, NAD83 projection.
gdf = gpd.GeoDataFrame(wells, geometry=gpd.points_from_xy(x=wells.LONGITUDE, y=wells.LATITUDE), crs='EPSG:4326')

# Reproject to UTM 11N.
gdf = gdf.to_crs('EPSG:26911')


gdf.to_file(results_path / 'wells.shp'.format(county))

## **Sample Groups**

### Create Sample Groups

In [ ]:
# Group samples by WID and LOGDATE apply list function to get list of PARLABELS for each group.
sample_groups = samples_mcl.groupby(['WID', 'LOGDATE'])['PARLABEL'].apply(list)

### Single Contaminant List

In [ ]:
# Use list comprehension to create a list of sample indexes where all contaminants in the contaminant list are present.
index_list = [i for i in sample_groups.index if all(item in sample_groups.loc[i] for item in contaminants_3)]

# Uses index_list to create a dataframe of samples that meet the criteria.
sample_group_results = samples_mcl[samples_mcl['GID'].isin(index_list)]

In [ ]:
# Print groups of samples that meet the criteria.
print('Groups: ',len(index_list))
print('Samples: ',len(sample_results))

In [ ]:
# Join location data to sample results.
sample_group_results = sample_results.merge(wells, left_on='WID', right_on='WID', how='inner')

# Save sample group results to csv.
sample_group_results.to_csv(bp / '{}_sample_results.csv'.format(county.lower()))

In [ ]:
sample_results.to_csv(bp / '{}_sample_results.csv'.format(county.lower()))

### Contaminant Combinations

In [ ]:
from itertools import combinations

# Create list of all combinations of all contaminants
combinations_list = list(combinations(contaminants_3, 10))
len(combinations_list)

In [ ]:
# Function to select sample groups based on combinations of contaminants.
def get_select_samples(row, contaminants):

    # checks list of contaminants in row against list of contaminants in function call.
    # if all contaminants in row are in contaminants, return True.
    if all(item in row for item in contaminants):
        return True

    else:
        return False

In [ ]:
def get_select_samples(row, contaminants):

    count = 0

    for values in row:
        print(values)
        if all(item in values for item in contaminants):
            count += 1
    return count

In [ ]:
ser_dict = {}

total =  len(combinations_list)
count = 0

for contaminants in combinations_list:

    count += 1
    percent = int(((count/total)*100))

    ser = sample_groups.apply(get_select_samples, contaminants=contaminants_3)

    ser_dict[contaminants] = ser

    #print('{}%'.format(percent))
    
combo_stats = pd.DataFrame.from_dict(ser_dict, orient='index')

print(combo_stats.max())
print(list(combo_stats.idxmax()))

In [ ]:
ser_dict

## **Modin Combo Stats**

In [ ]:
def get_select_samples_modin(row, contaminants):
    print(row)

    if all(element in row for element in contaminants) ==  True:
        print('contains all elements')
        return True

    else:
        print('does not contain all elements')
        return False

In [ ]:
combinations_list

In [ ]:
samples_modin = mpd.DataFrame(samples)

In [ ]:
import modin.pandas as mpd
from distributed import Client
client = Client()

sample_groups_modin = mpd.DataFrame(sample_groups)


ser_dict = {}

for contaminants in combinations_list:

    ser = sample_groups_modin.apply(get_select_samples_modin, contaminants=contaminants)
    ser = ser[ser == True]

    ser_dict[contaminants] = len(ser)



print(max(ser_dict.values()))

In [ ]:
print(combo_stats.max())
print(list(combo_stats.idxmax()))

In [ ]:
df[df == True]

In [ ]:
df[df == True]

In [ ]:
print('Loading MCL table \n')

# Create path to mcl table.
mcl_path = bp / 'MCL_list_1.xlsx'

# Open mcl table.
mcl = pd.read_excel(mcl_path, engine='openpyxl')

# join MCL values to sample results
print('Joining MCL values to samples \n')
samples_mcl = select_samples.merge(mcl, left_on='PARLABEL', right_on='chem_abrv', how='left').set_index(select_samples.index)

In [ ]:
# Save samples_mcl to csv.
alt = input("Input filename ending for 'county'_select_samples_'input'.csv: ")
name = '{}_select_samples_{}.csv'.format(county.lower(), alt)
sp = bp / name
samples_mcl.to_csv(sp)

In [ ]:
# Get counts of samples for each contaminant.
parlabel_stats = samples['PARLABEL'].value_counts()

# Create a dataframe with the counts of samples for each contaminant.
parlabel_stats = parlabel_stats.to_frame(name='COUNTS').reset_index().rename(columns={'index':'PARLABEL'})

# Create PERCENT column for each contaminant. Showing percent of samples for each contaminant compared to total samples.
parlabel_stats['PERCENT'] = (parlabel_stats['COUNTS'] / len(samples) * 100).round(4)

In [ ]:
# Save samples_mcl to csv.
name = '{}_parlabel_stats.csv'.format(county.lower())
sp = bp / name
parlabel_stats.to_csv(sp)